In [12]:
from ansys.mapdl.core import launch_mapdl
import numpy as np
import matplotlib.pyplot as plt

In [43]:
# MAPDL starten
mapdl = launch_mapdl(nproc=1)
# Start mapdl and clear it.
mapdl.clear()

# Enter verification example mode and the pre-processing routine.
mapdl.prep7()

*****MAPDL VERIFICATION RUN ONLY*****
     DO NOT USE RESULTS FOR PRODUCTION

          ***** MAPDL ANALYSIS DEFINITION (PREP7) *****

### Define element type

In [44]:
# Type of analysis: static.
mapdl.antype("STATIC")

# Element type: BEAM188.
mapdl.et(1, "188")
mapdl.keyopt(1, 5, 1)   # 2D problem (uy, ux and rotz)

ELEMENT TYPE       1 IS BEAM188      2-D 2-NODE BEAM             
  KEYOPT( 1- 6)=        0      0      0        0      1      0
  KEYOPT( 7-12)=        0      0      0        0      0      0
  KEYOPT(13-18)=        0      0      0        0      0      0

 CURRENT NODAL DOF SET IS  UX    UY    ROTZ
  TWO-DIMENSIONAL MODEL

### Define material

In [45]:
mapdl.mp("EX", 1, 2.1e11)       # youngs modulus N/m^2
mapdl.mp("PRXY", 1, 0.3)        # Poisson ratio

MATERIAL          1     PRXY =  0.3000000

### Define section

In [46]:
mapdl.sectype(1, "Beam", "RECT", "my_sec")
mapdl.secdata(0.2, 0.5) # b and h in m

SECTION ID NUMBER IS:            1
   BEAM SECTION TYPE IS:     Rectangle       
   BEAM SECTION NAME IS:     my_sec  
   COMPUTED BEAM SECTION DATA SUMMARY:
    Area                 = 0.10000    
    Iyy                  = 0.20833E-02
    Iyz                  = 0.54210E-19
    Izz                  = 0.33333E-03
    Warping Constant     = 0.35809E-05
    Torsion Constant     = 0.10213E-02
    Centroid Y           = 0.18974E-17
    Centroid Z           = 0.43368E-17
    Shear Center Y       =-0.14129E-15
    Shear Center Z       = 0.35099E-16
    Shear Correction-xy  = 0.84211    
    Shear Correction-yz  = 0.11341E-15
    Shear Correction-xz  = 0.84211    
                 
    Beam Section is offset to CENTROID of cross section

### Define nodes and elements

In [47]:
# Define nodes
for node_num in range(1, 6):
    mapdl.n(node_num, (node_num - 1) * 120, 0, 0)

# Define one node for the orientation of the beam cross-section.
orient_node = mapdl.n(6, 60, 1)

# Define one node for the orientation of the beam cross-section.
orient_node = mapdl.n(6, 60, 1)

# Define elements
for elem_num in range(1, 5):
    mapdl.e(elem_num, elem_num + 1, orient_node)

print(mapdl.nlist())
print(mapdl.elist())

LIST ALL SELECTED NODES.   DSYS=      0
   *****MAPDL VERIFICATION RUN ONLY*****
     DO NOT USE RESULTS FOR PRODUCTION

    NODE        X             Y             Z           THXY     THYZ     THZX
        1   0.0000        0.0000        0.0000          0.00     0.00     0.00
        2   120.00        0.0000        0.0000          0.00     0.00     0.00
        3   240.00        0.0000        0.0000          0.00     0.00     0.00
        4   360.00        0.0000        0.0000          0.00     0.00     0.00
        5   480.00        0.0000        0.0000          0.00     0.00     0.00
        6   60.000        1.0000        0.0000          0.00     0.00     0.00
LIST ALL SELECTED ELEMENTS.  (LIST NODES)
   *****MAPDL VERIFICATION RUN ONLY*****
     DO NOT USE RESULTS FOR PRODUCTION
 
    ELEM MAT TYP REL ESY SEC        NODES
 
       1   1   1   1   0   1      1     2
       2   1   1   1   0   1      2     3
       3   1   1   1   0   1      3     4
       4   1   1   1   0   1    

### Define boundary conditions

In [48]:
# BC for the beams seats
mapdl.d(1, "UX")
mapdl.d(2, "UY")
mapdl.d(5, "UY")

SPECIFIED CONSTRAINT UY   FOR SELECTED NODES            5 TO           5 BY           1
 REAL=  0.00000000       IMAG=  0.00000000

### Define force

In [49]:
mapdl.f(3, "FY", -100)

SPECIFIED NODAL LOAD FY   FOR SELECTED NODES         3 TO        3 BY        1
  REAL= -100.000000       IMAG=  0.00000000

### Solve

In [50]:
mapdl.run("/SOLU")
out = mapdl.solve()
mapdl.finish()

MapdlExitedError: Error:
MAPDL server connection terminated unexpectedly while running:
  SOLVE,
called by:
  _send_command

Suggestions:
  MAPDL *might* have died because it executed a not-allowed command or ran out of memory.
  Check the MAPDL command output for more details.
  Open an issue on GitHub if you need assistance: https://github.com/ansys/pymapdl/issues
MAPDL instance:
  C:/Users/FJ/AppData/Local/Temp/ansys_ugmwudcaij
  127.0.0.1:50053
Error:
  failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:50053: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.
 -- 10061)
Full error:
<_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNAVAILABLE: ipv4:127.0.0.1:50053: ConnectEx: Connection refused (No connection could be made because the target machine actively refused it.
 -- 10061)"
	debug_error_string = "UNKNO

### Post processing

In [ ]:
# Enter the post-processing routine and select the first load step.
mapdl.post1()
mapdl.set(1)

# 
uy = mapdl.get_value(entity="NODE", entnum=4, item1="u", it1num="x")

print(uy)

0.0


In [53]:
mapdl.exit()